<a href="https://colab.research.google.com/github/pietromarini00/Semantic_Parsing_Multilingual_Extension_CS4650/blob/main/CS4650_FinalProject_readable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Setup

## Imports

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from datasets import load_dataset, Split

import torch
from torch.nn.utils.rnn import pad_sequence

from torch.utils.data import Dataset, DataLoader
from functools import partial

from torch import optim
from transformers import get_linear_schedule_with_warmup
import time

import torch.nn as nn

import tqdm

## Load pretrained model

In [ ]:
sql_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
sql_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

In [ ]:
sql_model

## Tests with pretrained model

In [ ]:
#train_dataset  = load_dataset('wikisql', split=Split.TRAIN)
#valid_dataset = load_dataset('wikisql', split=Split.VALIDATION)
#test_dataset = load_dataset('wikisql', split=Split.TEST)
#torch.save(train_dataset, "train.csv")

In [ ]:
"""
# this is used for saving the english2sql
print(train_dataset)
import pandas as pd
train = []
for i in range(5000):
  a = {}
  a['question'] = train_dataset[i]['question']
  a['sql'] = train_dataset[i]['sql']['human_readable']
  train.append(a)
print(train)
import csv
keys = train[0].keys()

with open('train.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys, delimiter='|')
    dict_writer.writeheader()
    dict_writer.writerows(train)
"""

In [ ]:
def get_sql(query):
  input_text = "translate English to SQL: %s </s>" % query
  features = sql_tokenizer([input_text], return_tensors='pt')

  output = sql_model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  return sql_tokenizer.decode(output[0])

query = "What is the highest performance of model BERT?"

get_sql(query)

In [ ]:
"""
for i in [0, 50, 100, 200, 500, 1000]:
  print("Question:", valid_dataset[i])
  print("Answer:", valid_dataset[i]['sql']['human_readable'])
  print("Prediction:", get_sql(valid_dataset[i]['question']))
  print("============================================================================================================================")
"""

# First Model: Translate and Transform

In [ ]:
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

In [ ]:
def get_translation(text):
  features = translation_tokenizer([text], return_tensors='pt')

  output = translation_model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  return translation_tokenizer.decode(output[0])

#query = "What is the highest performance of a BERT model?"
new_query = "Was ist die hoechste performance von einem model BERT"

get_translation(new_query)

In [ ]:
def get_german_sql(german_query):
  features = translation_tokenizer([german_query], return_tensors='pt')

  output = translation_model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  english_query = translation_tokenizer.decode(output[0])

  #print(english_query)

  input_text_english = "translate English to SQL: %s </s>" % english_query
  #print(input_text_english)
  features = sql_tokenizer([input_text_english], return_tensors='pt')

  output = sql_model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  return sql_tokenizer.decode(output[0])

get_german_sql("Auf welche Schule ist der Spieler gegangen, der von 2010-2012 in Toronto war?")

# Second Model: Retrain


## Setup Dataset

In [ ]:
import pandas as pd
#load the data into a pandas dataframe
full_df = pd.read_csv('train_german.csv', sep='; ', header=0)

In [ ]:
#divide data into train, validation, and test datasets
num_queries = len(full_df)
idxs = list(range(num_queries))
print('Total queries in dataset: ', num_queries)
test_idx = idxs[:int(0.1*num_queries)]
val_idx = idxs[int(0.1*num_queries):int(0.2*num_queries)]
train_idx = idxs[int(0.2*num_queries):]

train_df = full_df.iloc[train_idx].reset_index(drop=True)
val_df = full_df.iloc[val_idx].reset_index(drop=True)
test_df = full_df.iloc[test_idx].reset_index(drop=True)

train_data = train_df[['question', 'sql']]
val_data   = val_df[['question', 'sql']]
test_data  = test_df[['question', 'sql']]

In [ ]:
#Defining torch dataset class for disaster tweet dataset
class SQLDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return self.df.iloc[idx]

In [ ]:
#set up train, validation, and testing datasets
train_dataset = SQLDataset(train_data)
val_dataset   = SQLDataset(val_data)
test_dataset  = SQLDataset(test_data)

## Model

In [ ]:
sql_model_retrain = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
sql_tokenizer_retrain = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

In [ ]:
#define hyperparameters
BATCH_SIZE = 20
LR = 1e-5
WEIGHT_DECAY = 0
N_EPOCHS = 5
CLIP = 1.0

#define models, move to device, and initialize weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = sql_model_retrain
model.to(device)
print('Model Initialized')

In [ ]:
def transformer_collate_fn(batch, tokenizer):

  sentences, labels, masks = [], [], []
  i = 0
  for data in batch:
    i += 1
    #print(data['question'])
    #print(sql_tokenizer([data['question']]))
    #print(en_de_translator(data['question'])[0]['translation_text'])
    tokenizer_output = sql_tokenizer([data['question']])
    #print(tokenizer_output)
    label_output = sql_tokenizer([data['sql']])
    tokenized_sent = tokenizer_output['input_ids'][0]
    label_sent = label_output['input_ids'][0]
    mask = tokenizer_output['attention_mask'][0]
    sentences.append(torch.tensor(tokenized_sent))
    labels.append(torch.tensor(label_sent))
    masks.append(torch.tensor(mask))
  sentences = pad_sequence(sentences, batch_first=True, padding_value=0)
  labels = pad_sequence(labels, batch_first=True, padding_value=0)
  masks = pad_sequence(masks, batch_first=True, padding_value=0.0)
  return sentences, labels, masks

In [ ]:
# load multilingual tokenizer
from transformers import T5Tokenizer

ml_tokenizer = T5Tokenizer.from_pretrained("google/mt5-large")

In [ ]:
#create pytorch dataloaders from train_dataset, val_dataset, and test_datset

train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=ml_tokenizer), shuffle = True)
val_dataloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=ml_tokenizer))
test_dataloader = DataLoader(test_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=ml_tokenizer))

In [ ]:
#train a given model, using a pytorch dataloader, optimizer, and scheduler (if provided)
def train(model,
          dataloader,
          optimizer,
          device,
          clip: float,
          epoch,
          scheduler = None):

    model.train()
    crit = nn.CrossEntropyLoss(ignore_index=0)
    epoch_loss = 0

    with tqdm.notebook.tqdm(
                dataloader,
                desc="epoch {}".format(epoch + 1),
                unit="batch",
                total=len(dataloader)) as batch_iterator:
      for batch in batch_iterator:
          sentences, labels, masks = batch[0], batch[1], batch[2]

          optimizer.zero_grad()

          output = model(input_ids=sentences.to(device), decoder_input_ids=labels.to(device))
          loss = crit(output.logits.transpose(2, 1), labels.to(device))
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

          optimizer.step()
          if scheduler is not None:
            scheduler.step()

          epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [ ]:
# evaluate loss
def evaluate(model,
             dataloader,
             device):

    model.eval()
    crit = nn.CrossEntropyLoss(ignore_index=0)

    epoch_loss = 0
    with torch.no_grad():
      with tqdm.notebook.tqdm(
                dataloader,
                desc="Eval",
                unit="batch",
                total=len(dataloader)) as batch_iterator:
        for batch in batch_iterator:
            sentences, labels, masks = batch[0], batch[1], batch[2]
            output = model(input_ids=sentences.to(device), decoder_input_ids=labels.to(device))
            loss = crit(output.logits.transpose(2, 1), labels.to(device))

            epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [ ]:
#calculate the prediction accuracy on the provided dataloader
def evaluate_acc(model,
                 dataset,
                 device):

    model.eval()


    epoch_loss = 0
    with torch.no_grad():
      total_correct = 0
      position_correct = 0
      total = 0
      with tqdm.notebook.tqdm(
                dataset,
                desc="Eval Acc German2SQL",
                unit="batch",
                total=len(dataset)) as batch_iterator:
        for batch in batch_iterator:

            sentences, labels = batch[0], batch[1]
            #print(sentences)
            input_text = "translate to SQL: %s </s>" % sentences
            features = sql_tokenizer_retrain([input_text], return_tensors='pt')
            output = sql_model_retrain.generate(input_ids=features['input_ids'].to(device),
                          attention_mask=features['attention_mask'].to(device))
            output = sql_tokenizer_retrain.decode(output[0])
            output_list = output.replace("</s>", "").lower().split()[1:]
            #print(output_list)
            label_list = labels.lower().split()
            #print(label_list)
            for i, word in enumerate(label_list):
              total += 1
              if word in output_list:
                total_correct += 1
              try:
                index = output_list.index(word)
                if index == i:
                  position_correct += 1
              except:
                pass

    return total_correct / total#, position_correct / total

In [ ]:
#count the number of trainable parameters in the model
def count_parameters(model: torch.nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
#computes the amount of time that a training epoch took and displays it in human readable form
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LR)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=N_EPOCHS*len(train_dataloader))

print(f'The model has {count_parameters(model):,} trainable parameters')



# get example query
query = "Wie hoch ist die Gesamtzahl der Episoden, deren Erstausstrahlung von 1,82 Millionen Zuschauern gesehen wurde?"
input_text = "translate to SQL: %s </s>" % query
features = sql_tokenizer_retrain([input_text], return_tensors='pt')
output = sql_model_retrain.generate(input_ids=features['input_ids'].to(device),
              attention_mask=features['attention_mask'].to(device))
print(sql_tokenizer_retrain.decode(output[0]))

train_loss = evaluate(model, train_dataloader, device)
#train_acc = evaluate_acc(model, train_dataset, device)
valid_loss = evaluate(model, val_dataloader, device)
#valid_acc = evaluate_acc(model, val_dataset, device)

print(f'Initial Train Loss: {train_loss:.8f}')
#print(f'Initial Train Acc: {train_acc:.8f}')
print(f'Initial Valid Loss: {valid_loss:.8f}')
#print(f'Initial Valid Acc: {valid_acc:.8f}')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_dataloader, optimizer, device, CLIP, epoch, scheduler)
    end_time = time.time()
    #train_acc = evaluate_acc(model, train_dataset, device)
    valid_loss = evaluate(model, val_dataloader, device)
    #valid_acc = evaluate_acc(model, val_dataset, device)
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.8f}')
    #print(f'\tTrain Acc: {train_acc:.8f}')
    print(f'\tValid Loss: {valid_loss:.8f}')
    #print(f'\tValid Acc: {valid_acc:.8f}')
    # get example query
    features = sql_tokenizer_retrain([input_text], return_tensors='pt')
    output = sql_model_retrain.generate(input_ids=features['input_ids'].to(device),
                  attention_mask=features['attention_mask'].to(device))
    print(sql_tokenizer_retrain.decode(output[0]))

test_loss = evaluate(model, test_dataloader, device)
#test_acc = evaluate_acc(model, test_dataset, device)
print(f'Test Loss: {test_loss:.8f}')
#print(f'\tTest Acc: {test_acc:.8f}')
# get example query
features = sql_tokenizer_retrain([input_text], return_tensors='pt')
output = sql_model_retrain.generate(input_ids=features['input_ids'].to(device),
              attention_mask=features['attention_mask'].to(device))
print(sql_tokenizer_retrain.decode(output[0]))

# Evaluation

In [ ]:
# calculate Accuracy of German2English2SQL
def evaluate_acc_tanslate_sql(dataloader,
                 device):


    epoch_loss = 0
    with torch.no_grad():
      total_correct = 0
      position_correct = 0
      total = 0
      with tqdm.notebook.tqdm(
                dataloader,
                desc="Eval Acc German2English2SQL",
                unit="batch",
                total=len(dataloader)) as batch_iterator:
        for batch in batch_iterator:
            sentences, labels = batch[0], batch[1]
            #print(sentences)
            output = get_german_sql(sentences)
            output_list = output.replace("</s>", "").lower().split()[1:]
            # print(output_list)
            label_list = labels.lower().split()
            # print(label_list)
            total_sentence = 0
            total_sentence_correct = 0
            for i, word in enumerate(label_list):
              total += 1
              total_sentence += 1
              if word in output_list:
                total_correct += 1
                total_sentence_correct += 1
              try:
                index = output_list.index(word)
                if index == i:
                  position_correct += 1
              except:
                pass
            # print(total_sentence_correct / total_sentence)

    return total_correct / total, position_correct / total

In [ ]:
evaluate_acc_tanslate_sql(test_dataset, device)

In [ ]:
#calculate accuracy of German2SQL
def evaluate_acc(model,
                 dataset,
                 device):

    model.eval()


    epoch_loss = 0
    with torch.no_grad():
      total_correct = 0
      position_correct = 0
      total = 0
      with tqdm.notebook.tqdm(
                dataset,
                desc="Eval Acc German2SQL",
                unit="batch",
                total=len(dataset)) as batch_iterator:
        for batch in batch_iterator:

            sentences, labels = batch[0], batch[1]
            #print(sentences)
            input_text = "translate to SQL: %s </s>" % sentences
            features = sql_tokenizer_retrain([input_text], return_tensors='pt')
            output = sql_model_retrain.generate(input_ids=features['input_ids'].to(device),
                          attention_mask=features['attention_mask'].to(device))
            output = sql_tokenizer_retrain.decode(output[0])
            output_list = output.replace("</s>", "").lower().split()[1:]
            # print(output_list)
            label_list = labels.lower().split()
            # print(label_list)
            total_sentence = 0
            total_sentence_correct = 0
            for i, word in enumerate(label_list):
              total += 1
              total_sentence += 1
              if word in output_list:
                total_correct += 1
                total_sentence_correct += 1
              try:
                index = output_list.index(word)
                if index == i:
                  position_correct += 1
              except:
                pass
            # print(total_sentence_correct / total_sentence)

    return total_correct / total, position_correct / total

In [ ]:
evaluate_acc(sql_model_retrain, test_dataset, device)